# ¿Por qué PyCUDA?

Hasta ahora hemos visto que si bien CUDA no es un lenguaje imposible de aprender, puede llegar a ser un dolor de cabeza el tener muchos apuntadores y manejar la memoria de un modo tan rudimentario.

Sin embargo hay alternativas que nos permiten trabajar en entornos más agradables, un ejemplo de ellos es [PyCUDA](http://mathema.tician.de/software/pycuda/) creado con [Andreas Klöckner](http://mathema.tician.de/). Básicamente PyCUDA se encarga de mapear todo CUDA dentro de Python. 

Por poner un ejemplo, un código simple sería el siguiente

In [1]:
import pycuda.autoinit
import pycuda.driver as drv
import numpy

from pycuda.compiler import SourceModule
mod = SourceModule("""
__global__ void multiplicar(float *dest, float *a, float *b)
{
  const int i = threadIdx.x;
  dest[i] = a[i] * b[i];
}
""")

multiplicar = mod.get_function("multiplicar")

a = numpy.random.randn(400).astype(numpy.float32)
b = numpy.random.randn(400).astype(numpy.float32)

dest = numpy.zeros_like(a)

print dest

multiplicar(
        drv.Out(dest), drv.In(a), drv.In(b),
        block=(400,1,1), grid=(1,1))

print dest

print dest-a*b

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0

Al correr este programa vamos a obtener un montón de ceros; algo no muy interesante. Sin embargo detrás de escenas sí pasó algo interesante.

- PyCUDA compiló el código fuente y lo cargó a la tarjeta.
- Se asignó memoria automáticamente, además de copiar las cosas de CPU a GPU y de vuelta.
- Por último la limpieza (liberación de memoria) se hace sola.

**Útil ¿cierto?**

# Usando PyCUDA

Para empezar debemos importar e incializar PyCUDA

In [2]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

## Transferir datos

El siguiente paso es transferir datos al GPU. Principalmente arreglos de numpy. Por ejemplo, tomemos un arreglo de números aleatorios de $4 \times 4$

In [3]:
import numpy
a = numpy.random.randn(4,4)

sin embargo nuestro arreglo `a` consiste en números de doble precisión, dado que no todos los GPU de NVIDIA cuentan con doble precisión es que hacemos lo siguiente

In [4]:
a = a.astype(numpy.float32)

finalmente, necesitmos un arreglo hacia el cuál transferir la información, así que deberíamos guardar la memoria en el dispositivo:

In [5]:
a_gpu = cuda.mem_alloc(a.nbytes)

como último paso, necesitamos tranferir los datos al GPU

In [6]:
cuda.memcpy_htod(a_gpu, a)

## Ejecutando kernels


Durante este capítulo nos centraremos en un ejemplo muy simple. Escribir un código para duplicar cada una de las entradas en un arreglo, en seguida escribimos el kernel en CUDA C, y se lo otorgamos al constructor de `pycuda.compiler.SourceModule`

In [7]:
mod = SourceModule("""
  __global__ void duplicar(float *a)
  {
    int idx = threadIdx.x + threadIdx.y*4;
    a[idx] *= 2;
  }
  """)

Si no hay errores, el código ahora ha sido compilado y cargado en el dispositivo. Encontramos una referencia a nuestra `pycuda.driver.Function` y la llamamos, especificando `a_gpu` como el argumento, y un tamaño de bloque de $4\times 4$:

In [8]:
mod

In [9]:
func = mod.get_function("duplicar")
func(a_gpu, block=(4,4,1))

In [10]:
func

Finalmente recogemos la información del GPU y la mostramos con el `a` original

In [11]:
a_duplicado = numpy.empty_like(a)
cuda.memcpy_dtoh(a_duplicado, a_gpu)
print a_duplicado
print a

[[-2.97226572  0.20464714  2.41393161 -2.09682822]
 [-1.63446426  0.7785539   1.59411359  0.42689195]
 [-1.14897263 -0.54843307  4.48510981  0.04625186]
 [-0.64690572  2.06020832 -1.02194703  5.90874815]]
[[-1.48613286  0.10232357  1.2069658  -1.04841411]
 [-0.81723213  0.38927695  0.79705679  0.21344598]
 [-0.57448632 -0.27421653  2.2425549   0.02312593]
 [-0.32345286  1.03010416 -0.51097351  2.95437407]]


In [12]:
print(type(a))
print(type(a_gpu))
print(type(a_duplicado))

<type 'numpy.ndarray'>
<class 'pycuda._driver.DeviceAllocation'>
<type 'numpy.ndarray'>


In [13]:
a_duplicado

array([[-2.97226572,  0.20464714,  2.41393161, -2.09682822],
       [-1.63446426,  0.7785539 ,  1.59411359,  0.42689195],
       [-1.14897263, -0.54843307,  4.48510981,  0.04625186],
       [-0.64690572,  2.06020832, -1.02194703,  5.90874815]], dtype=float32)

¡Y eso es todo! Hemos terminado con el trabajo. PyCUDA se encarga de hacer toda la limpieza por nosotros.